In [1]:
search_for = 131
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.06482982635498047
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 296765305
type: [1, 1, 1, 1, 131] 131
cases of this type: 294499921
100000 8.192975729705164
200000 11.046928642268181
300000 10.687221162490111
400000 11.886041399447343
500000 13.270420451149642
600000 16.89085937490758
700000 18.33791493422535
800000 20.97299545184857
900000 21.5889171242653
1000000 25.490174064225027
1100000 22.369250670112834
1200000 12.071836003400097
1300000 25.697625949186804
1400000 22.539709088709877
1500000 25.69991008710381
1600000 26.41653928976327
1700000 26.49654416168425
1800000 23.378777071473095
1900000 26.4018477026684
2000000 26.977199920585367
2100000 24.629107558733168
2200000 24.952178088640093
2300000 7.183466764432599
2400000 29.579203028698178
2500000 32.480841795985846
2600000 26.5683971734267
2700000 25.6473432977523
2800000 27.117106189794434
2900000 25.515763369992865
3000000 25.81417

29500000 7.465387294193565
29600000 21.030106806027632
29700000 21.879541270347534
29800000 21.222199400162292
29900000 21.412631209952092
30000000 20.266488307792862
30100000 20.901934245018438
30200000 18.48556214324074
30300000 20.413596670917496
30400000 5.960450675793707
30500000 19.4657522521146
30600000 19.880001266291455
30700000 19.25535123385822
30800000 21.110878809123673
30900000 20.33654715336584
31000000 20.82869796885898
31100000 21.431207429294748
31200000 20.7421721373331
31300000 10.158813076061904
31400000 20.74533684776696
31500000 23.044214053293135
31600000 20.58502479137975
31700000 19.40068904730196
31800000 12.292099263176452
31900000 20.132960362134458
32000000 22.453276632996186
32100000 19.524956698801773
32200000 22.350165664023397
32300000 19.284291397173114
32400000 21.38802146047026
32500000 18.986519306875863
32600000 12.21322182133296
32700000 15.420064708598165
32800000 18.317933283748527
32900000 21.793412264911034
33000000 19.620513223742524
3310000

59200000 17.36167871210942
59300000 18.96111538040463
59400000 16.448748445995555
59500000 18.978660088995415
59600000 7.092760959272435
59700000 15.577375190728814
59800000 18.812967772105097
59900000 16.097947652787404
60000000 19.416977686579923
60100000 17.25372295046568
60200000 19.763419759854308
60300000 8.094397299884763
60400000 18.797411382744478
60500000 21.31556204393314
60600000 18.19275151113862
60700000 20.963824902426573
60800000 18.848497365679773
60900000 20.039013774751055
61000000 19.254334020814
61100000 19.058396615029473
61200000 8.527380462751369
61300000 16.168746749111026
61400000 19.18891933368591
61500000 17.16275674844885
61600000 17.98602435867591
61700000 16.809712151646146
61800000 15.306890690746519
61900000 8.464634436849385
62000000 16.120820562434425
62100000 17.8738825223167
62200000 16.449421514651487
62300000 17.856928195746658
62400000 17.995383767017234
62500000 9.163251667466302
62600000 18.097530968390252
62700000 18.408627267361666
62800000 2

89000000 15.700442928829622
89100000 13.98677021294447
89200000 14.651377663630745
89300000 7.333020456418866
89400000 14.892295752096356
89500000 15.995737931771822
89600000 15.414055891418176
89700000 15.513151490531202
89800000 16.74340686351418
89900000 16.455344246794997
90000000 16.988568557417675
90100000 15.862547818283508
90200000 16.97920704995564
90300000 15.712356695403953
90400000 16.552349878776205
90500000 14.258446559022303
90600000 13.423510387554193
90700000 8.341336262084514
90800000 13.996151843471255
90900000 15.187002148829324
91000000 14.261783978179745
91100000 4.059470818156209
91200000 16.235880002922706
91300000 14.521141274425258
91400000 15.138953104880017
91500000 8.046363924887222
91600000 12.892915116421223
91700000 15.567388706763161
91800000 15.071544206078327
91900000 16.075423034770548
92000000 15.890215581606336
92100000 15.651567557477758
92200000 16.410514615798974
92300000 15.618786739223832
92400000 16.386709473188365
92500000 15.203750897976816

118000000 11.298603321995328
118100000 5.888340530162781
118200000 12.092431329159611
118300000 6.172034246936401
118400000 12.864338765057035
118500000 12.27957907439897
118600000 13.253554225617975
118700000 12.705495312391372
118800000 13.547692039387181
118900000 12.73445025404225
119000000 13.99029633440812
119100000 13.865869544371304
119200000 15.086811776831636
119300000 13.200146952435245
119400000 13.22129317044717
119500000 12.85141336414045
119600000 11.872627700485339
119700000 13.200618861992641
119800000 11.585763130952596
119900000 13.462941056693593
120000000 11.435115890817798
120100000 5.585061197272261
120200000 12.433888210823746
120300000 4.844492087122391
120400000 10.990245054122179
120500000 6.559953170031981
120600000 11.609046831944694
120700000 11.608752741774936
120800000 12.965492310009129
120900000 12.038626405202244
121000000 11.561196977926741
121100000 13.31939312695845
121200000 12.198515971570144
121300000 14.140974326794893
121400000 13.106604976481

...

175600000 9.849332410109628
175700000 8.724310433164218
175800000 9.486966745843171
175900000 8.42030354123487
176000000 9.437783173319668
176100000 8.19720025222754
176200000 9.021951630153925
176300000 4.227362439769076
176400000 8.380090809004573
176500000 3.5079527950292424
176600000 7.876443179635316
176700000 6.001187016962289
176800000 5.965857960886875
176900000 9.483311505823218
177000000 8.542184566893406
177100000 9.07481818416646
177200000 8.91884069959573
177300000 8.71634719549886
177400000 9.21403158994664
177500000 9.30103489465281
177600000 9.815582060337407
177700000 9.282052796760558
177800000 9.660906036381565
177900000 8.850680121320247
178000000 8.822971714218646
178100000 8.391608906181244
178200000 8.969844734251257
178300000 7.969941644215346
178400000 8.764880595082905
178500000 5.274451985937949
178600000 6.963943418571233
178700000 6.7746589911376
178800000 4.390627389673359
178900000 7.8589376630134
179000000 3.680808130607062
179100000 8.915448489118171
17

204900000 6.76860904035256
205000000 7.167115891220212
205100000 6.506171491582896
205200000 6.926901913274308
205300000 2.7017597521446044
205400000 6.534118295674979
205500000 6.689965480369234
205600000 6.242384811609874
205700000 4.381346893267572
205800000 4.585593118780599
205900000 6.222048343202412
206000000 6.801203927345366
206100000 6.337587930784789
206200000 2.396738586293618
206300000 6.966678355315698
206400000 6.4264954918601855
206500000 7.131082974231105
206600000 6.856719674612429
206700000 7.4730310279182826
206800000 7.084042576446027
206900000 7.1222874919827674
207000000 7.106040871539073
207100000 6.89589319943144
207200000 6.875590091883984
207300000 6.640935531063599
207400000 6.7844600477199455
207500000 3.8315649660176434
207600000 5.258772162168731
207700000 6.584239950080753
207800000 6.1112505743262675
207900000 6.884906723194569
208000000 1.7575583937846622
208100000 6.532867027803262
208200000 6.510544179039978
208300000 6.147401280709923
208400000 5.00

234100000 4.789566670334866
234200000 5.000862366615409
234300000 1.8695516325184438
234400000 4.676243376074949
234500000 4.6373422980022
234600000 4.252811630359159
234700000 4.449351706003278
234800000 4.27673855591015
234900000 3.878248602504459
235000000 2.0721130952261686
235100000 4.231499914621062
235200000 4.440166669662724
235300000 4.170668950650036
235400000 4.48324749691832
235500000 4.569967538473185
235600000 3.155456487821311
235700000 3.515611169722895
235800000 4.672628394354823
235900000 4.890637966626962
236000000 4.72640222946914
236100000 4.985639652324017
236200000 4.886673345487237
236300000 4.7730394828559914
236400000 5.031941231922173
236500000 2.8782528365373317
236600000 3.852274877276791
236700000 4.520145028410282
236800000 4.419029055989156
236900000 4.456043902622356
237000000 4.1959282286857
237100000 4.49423521804895
237200000 1.84380950214535
237300000 4.253795726111332
237400000 4.6874645580985845
237500000 4.630416933256139
237600000 5.038772368722

263100000 2.6440569122566284
263200000 2.84451512883993
263300000 1.0847267954088815
263400000 2.8705358869222866
263500000 2.8026349632808865
263600000 2.5449642952798177
263700000 2.807196799538447
263800000 2.4156665933440027
263900000 2.571562659707033
264000000 2.3854984069045653
264100000 2.418375427541822
264200000 0.7155960078548292
264300000 2.497366933034132
264400000 2.5284497274937032
264500000 2.491814904233933
264600000 2.5815651812073255
264700000 2.6656972580864826
264800000 2.556234242582199
264900000 2.7092735023712518
265000000 2.643443605289986
265100000 0.9072901865426302
265200000 2.584524210803724
265300000 2.597791089622081
265400000 2.7237030254542764
265500000 1.5814293853738042
265600000 2.0988079803376096
265700000 2.580168290276213
265800000 2.6525051494483947
265900000 2.5215978676243527
266000000 2.4419504890737893
266100000 2.5063705908994414
266200000 2.3479064894349575
266300000 2.3386557661989196
266400000 1.4186065615602004
266500000 1.58310504032641

291600000 0.29260461540360583
291700000 0.2930329023472071
291800000 0.30382544033066433
291900000 0.37360550625829236
292000000 0.29193576761245726
292100000 0.29207754737287095
292200000 0.29327819241677394
292300000 0.05788319691872596
292400000 0.23518601049279503
292500000 0.241078529067364
292600000 0.24706621906561654
292700000 0.23091363123525516
292800000 0.2227057740432852
292900000 0.23793312229721744
293000000 0.22121517833125923
293100000 0.1900433443958163
293200000 0.16374203921622038
293300000 0.15511624334872434
293400000 0.08978155659194124
293500000 0.15919334752219916
293600000 0.2053722401947412
293700000 0.17071427908477188
293800000 0.1916994284719626
293900000 0.11307097180335388
294000000 0.1386517683398856
294100000 0.1319673495103419
294200000 0.12106721335747837
294300000 0.08412022976504101
294400000 0.1367598209465444
best so far: 0
type: [1, 1, 1, 131, 1] 131
cases of this type: 2248091
294500000 0.0907450944420331
294600000 0.013245003299196562
294700000